# Transforming SFHAs

This notebook experiments with transforming data from REST API queries in-place, either in pandas dataframes or python dicts.

In [ ]:
from arcgis.gis import GIS
from arcgis.features import Feature, FeatureSet, FeatureLayer, FeatureLayerCollection, SpatialDataFrame
from arcgis.geometry import Geometry, filters, union, buffer

Extract Boulder city limits, FEMA lomr boundaries, and FEMA special flood hazard areas (sfha) polygons

In [ ]:
city_lims_url = "https://maps.bouldercolorado.gov/arcgis/rest/services/plan/CityLimits/MapServer/0"
city = FeatureLayer(city_lims_url)

nfhl_url = "https://hazards.fema.gov/gis/nfhl/rest/services/public/NFHL/MapServer"
nfhl = FeatureLayerCollection(nfhl_url)
lomr = nfhl.layers[1]
sfha = nfhl.layers[27]

Define the output spatial reference

In [ ]:
sr = 2876 # NAD83(HARN) / Colorado North (ftUS)

Extract city limits as a spatial filter object, and get all lomrs inside

In [ ]:
# using the unioned city boundary polygon
anon_gis = GIS()
city_lims = city.query(out_sr=sr)
city_geoms = [poly.geometry for poly in city_lims.features]
city_union = union(spatial_ref=sr, geometries=city_geoms, gis=anon_gis)
geom_filter = filters.intersects(city_union, sr=sr)

The most recent CRS audit required the city to report on LOMRs after August 16, 2018. So the result we want is all LOMRs on or after this date that reside within the city's limits.

In [ ]:
date_str = '2020-02-01'
clause = f"STATUS = 'Effective' AND EFF_DATE >= '{date_str}'"

In [ ]:
boulder_lomrs = lomr.query(where=clause,
                           geometry_filter=geom_filter,
                           out_sr=sr)
# Make sure to drop LOMR polygons that have duplicate Case Numbers and Geometries.
temp = boulder_lomrs.sdf
temp['GEOM_STR'] = str(temp['SHAPE'])
temp.drop_duplicates(subset=['CASE_NO', 'GEOM_STR'], inplace=True)
temp.sort_values(by='EFF_DATE', inplace=True, ascending=False)
boulder_lomrs = FeatureSet.from_dataframe(temp)

In [ ]:
# All floodplain areas in Boulder County
flood_areas = sfha.query(where="DFIRM_ID = '08013C'",
                         out_fields=['FLD_AR_ID', 'STUDY_TYP', 'FLD_ZONE', 'ZONE_SUBTY', 'SFHA_TF', 'STATIC_BFE', 'DEPTH'],
                         out_sr=sr)

In [ ]:
# Create an empty dataframe as a container for all rows in `flood_areas` that are within the LOMR areas.
all_flood = flood_areas.sdf
flood_subset = all_flood.copy()
flood_subset.drop(list(range(len(flood_subset))),inplace=True)

In [ ]:
for l in boulder_lomrs.features:
    g = Geometry(l.geometry)
    
    # buffer LOMR geom by one foot to avoid topological
    # errors where polys share an edge
    buf = g.buffer(1)

    for row in flood_areas.features:
        area_id = row.attributes['FLD_AR_ID']
        f = Geometry(row.geometry)
        if buf.contains(f):
            flood_subset = flood_subset.append(all_flood[all_flood['FLD_AR_ID'] == area_id],
                                               ignore_index=True)

# drop any rows that represent duplicate flood areas
flood_subset.drop_duplicates(subset=['FLD_AR_ID'], inplace=True)

## Transformations

- FLOODPLAIN
  - 500-Year = "FLD_ZONE = 'X' AND ZONE_SUBTY = '0.2 PCT ANNUAL CHANCE FLOOD HAZARD'"
  - 100-Year = "SFHA_TF = 'T'"
    - Conveyance Zone = "SFHA_TF = 'T' AND 'ZONE_SUBTY' = 'FLOODWAY'"
- DRAINAGE
  - Use "set-theoretic" funcs: centroid inside city-floodplain with DRAINAGE = X
- LIFECYCLE = 'Active'
- ADOPTDATE = lomr enacted date. Scrape this during extract phase
- FEMAZONE
  - if FLD_ZONE = AO
    - FEMAZONE = FLD_ZONE + str(DEPTH)
  - if FLD_ZONE = AH
    - FEMAZONE = FLD_ZONE + str(STATIC_BFE)
- SOURCE = 'FEMA'
- INEFFDATE = '12/31/9999'
